# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,369.001028,0.008828,1234,369.009856,368.992200,738.019712,737.984401
6,152.057349,0.473915,1234,152.531264,151.583433,305.062528,303.166867
7,231.557864,0.662357,1234,232.220221,230.895508,464.440442,461.791015
8,482.740049,0.096489,1234,482.836539,482.643560,965.673077,965.287120
9,441.130680,-0.664097,1234,440.466582,441.794777,880.933165,883.589554
10,932.563242,-0.301851,1234,932.261391,932.865092,1864.522783,1865.730185
11,756.085247,0.152785,1234,756.238032,755.932462,1512.476065,1511.864924
12,623.852965,-0.956081,1234,622.896884,624.809046,1245.793769,1249.618092
13,831.653694,0.598232,1234,832.251926,831.055462,1664.503852,1662.110924
14,847.495264,-0.185502,1234,847.309762,847.680767,1694.619524,1695.361533


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-369.009856,368.992200
6,-152.531264,151.583433
7,-232.220221,230.895508
8,-482.836539,482.643560
9,-440.466582,441.794777
10,-932.261391,932.865092
11,-756.238032,755.932462
12,-622.896884,624.809046
13,-832.251926,831.055462
14,-847.309762,847.680767


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-369.009856,368.992200
6,-152.531264,151.583433
7,-232.220221,230.895508
8,-482.836539,482.643560
9,-440.466582,441.794777
10,-932.261391,932.865092
11,-756.238032,755.932462
12,-622.896884,624.809046
13,-832.251926,831.055462
14,-847.309762,847.680767


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-369.00985609,  368.9922003 ],
       [-152.5312642 ,  151.58343334],
       [-232.22022101,  230.89550765],
       [-482.83653869,  482.6435599 ],
       [-440.46658237,  441.79477677],
       [-932.26139138,  932.8650924 ],
       [-756.23803232,  755.93246206],
       [-622.89688447,  624.80904597],
       [-832.25192622,  831.05546198],
       [-847.30976217,  847.68076665]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,736.019712,-737.984401
6,303.062528,-303.166867
7,462.440442,-461.791015
8,963.673077,-965.287120
9,878.933165,-883.589554
10,1862.522783,-1865.730185
11,1510.476065,-1511.864924
12,1243.793769,-1249.618092
13,1662.503852,-1662.110924
14,1692.619524,-1695.361533


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
